In [3]:
import cv2
import numpy as np
from math import *
import matplotlib.pyplot as plt

In [4]:
# read an image

img_path = '../DRIVE/training/images/22_training.tif'
img = cv2.imread(img_path, 1)
img_shape = img.shape[:2]

In [32]:
cv2.imwrite('figures/km_dil_close.png', result)

True

In [25]:
cv2.imshow('result',result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# CREATING a MASK (ROI)

# thresholds determined empirically
T1 = 40; T2 = 50

# 1st criterium
avg_from_rgb = np.mean(img, axis=2)

# 2nd criterium
sum_of_diffs = np.empty(img.shape[:2])
mask = np.ones(img.shape[:2])

for i in range(img_shape[0]):
    for j in range (img_shape[1]):
        comb_rg = abs(img[i, j, 0] - img[i, j, 1])
        comp_rb = abs(img[i, j, 0] - img[i, j, 2])
        comb_gb = abs(img[i, j, 1] - img[i, j, 2])
        
        sum_of_diffs[i,j] = comb_rg + comp_rb + comb_gb
        
        if avg_from_rgb[i,j] < T1 and sum_of_diffs[i,j] < T2:
            mask[i,j] = 0

In [ ]:
# EXPANSION OF ROI (for avoiding artifacts from fundus image border)

In [9]:
# GRAY LEVEL CONVERSION according formula: c_r*R + c_g * G + c_b * B, where c_r = 0.1, c_g = 0.7, c_b = 0.2
img_gray = np.empty(img.shape[:2], dtype=np.uint8)
for i in range(img_shape[0]):
    for j in range(img_shape[1]):
        img_gray[i,j] = np.uint8(0.1 * img[i,j,0]) + np.uint8(0.7 * img[i,j,1]) + np.uint8(0.2 * img[i,j,2])

In [12]:
# VESSEL LIGHT REFLEX REMOVAL - using opening with 3 pixel diameter opening disc
from skimage.morphology import disk, opening
opened_img = opening(img_gray, disk(3))

In [15]:
# FEATURE EXTRACTION by using several vessel enhancement methods:
# 1. GABOR FILTER RESPONSE:
orientations = 24
base_angle = 12 

gabor_result_img = np.zeros_like(opened_img)
for i in range(orientations):
    theta = base_angle * (i+1)
    gabor_kern = cv2.getGaborKernel(ksize=(3,3), sigma=2, theta=theta, lambd=10, gamma=0.5)
    gabor_img = cv2.filter2D(opened_img, -1, gabor_kern)
    np.maximum(gabor_result_img, gabor_img, gabor_result_img) # getting maximum response

In [ ]:
# 2. GAUSSIAN METCHED-FILTER RESPONSE, 3. Frangi's multiscale vessel enhanement approach
gauss_result_img = cv2.GaussianBlur(opened_img, (5,5), 1)

In [19]:
# TOP-HAT for extracting small objects and details
from skimage.morphology import disk
tophat = cv2.morphologyEx(gabor_result_img, cv2.MORPH_TOPHAT, disk(5))

In [27]:
# K-Means
# K = 3 (3 regions of vesselness possiblity: high, medium, low)
z = np.reshape(tophat, (img_shape[0] * img_shape[1], 1))
z = np.float32(z)

criteria = (cv2.TERM_CRITERIA_EPS, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS

# Apply KMeans
compactness,labels,centers = cv2.kmeans(z,3,None,criteria,10,flags)

In [28]:
uniqueVals, counts = np.unique(labels, return_counts=True)
whiteLabel = np.argmin(counts)
blackLabel = np.argmax(counts)
grayLabel = [i for i in range(3) if i != whiteLabel and i != blackLabel]

In [29]:
centers = np.uint8(centers)
labels[labels==whiteLabel] = grayLabel[0]
res = centers[labels.flatten()]
result = res.reshape((img_shape))
a,b = np.unique(result, return_counts=True)
result[result==a[np.argmax(b)]] = 0
result[result==a[np.argmin(b)]] = 255

In [31]:
struct = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
result = cv2.dilate(result,struct,iterations = 1)
result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)))

In [ ]:
# PERFORMANCE
ground_truth_img_path = '../DRIVE/training/1st_manual/22_manual1.gif'
cap = cv2.VideoCapture(ground_truth_img_path)
ret, frame = cap.read()

In [ ]:
cv2.imshow('frame',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
tp = 0; tn = 0; fp = 0; fn = 0
for i in range(img_shape[0]):
    for j in range(img_shape[1]):
        px = result[i][j]
        if px == 0:
            if frame[i,j,1] == 0:
                tn = tn + 1
            elif frame[i,j,1] == 255:
                fn = fn + 1
        elif px == 255:
            if frame[i,j,1] == 0:
                fp = fp + 1
            elif frame[i,j,1] == 255:
                tp = tp + 1
                
sen = tp / (tp+fn) * 100 # sensitivity
spec = tn / (tn+fp) * 100 # specificity
acc = (tp + tn) / (tp + tn + fp + fn) * 100 # accuracy